### Stack
Stack is a `LIFO`, Last In First Out data structure. It is a list with restriction that insertion and deletion occurs only at one end.  
Stack can be implemented using arrays or using linked list.

### Array based Stack
```C++
#define MAX_SIZE = 10

int elements[MAX_SIZE];
int top = -1;

void push(int x){
    if(top==MAX_SIZE-1){
        cout<<"Overflow";
        return;
    }
    elements[++top] = x;
}

int pop(){
    if(top==-1){
        cout<<"Already empty";
        return;
    }
    top--;
    return elements[top + 1];
}

bool isEmpty(){
    if(top==-1){
        return true;
    }else{
        return false;
    }
}
```

We can modify the above code to extend the array when the array completely fills up. In such case the time complexities are:
1. push : Best case O(1), Worst case (when we have to extend) O(n), Average case O(1)
2. pop : O(1)

### Resizable Array
In the previous version, array's size was fixed. In the new version, we resize array as soon as the array is filled.
```C++
int arraySize = 10;
int elements[MAX_SIZE];
int top = -1;

void resize(int newSize){
    int newArray[newSize];
    for(int i=0; i<arraySize; i++){
        newArray[i] = elements[i];
    }
    
    elements = newArray;
    arraySize = newSize;
}

void push(int x){
    if(top==arraySize-1){
        resize(2*arraySize);
    }
    elements[++top] = x;
}

int pop(){
    if(top==-1){
        cout<<"Already empty";
        return;
    } else if(arraySize >= 3*top){
        resize(arraySize/2);
    }
    
    top--;
    return elements[top + 1];
}
```

### A Note on Time Complexity of Insertion of Elements in a Resizable Array
Whenever the array is full we double the size of array, copy the elements from old array to the new one. So what is the time complexity. To calculate the time complexity of addition of element into a resizable array, we do amortized time complexity which is basically weighted average time complexity. Suppose we have to insert $n$ elements into an array and we start with array having size $1$.  
So as soon as we add $1$ element, we create a new array with double the size and copy that $1$ element to the new array. When we add one more element, we double array and copy over $2$ elements. Adding the third element takes constant time. So for $n$ elements, the times are:  
$$1 \rightarrow 1$$
$$2 \rightarrow 2$$
$$3 \rightarrow 1$$
$$4 \rightarrow 4$$
$$5 \rightarrow 1$$
$$6 \rightarrow 1$$
$$\ldots$$
The time taken to insert $n$ elements,
$$T = (1+1+1+\ldots)_{K\ times} + (2^0 + 2^1 + 2^2 + \ldots)_{K'\ times}$$
Where
$$K+K'=n$$
And
$$2^{K'} \le n$$
$$K' \le log_2n$$
$$K' = log_2n + 1$$
Therefore,
$$T = K + \sum^{K'}_{0}2^i$$
$$T = n - log_2n - 1 + \sum^{K'}_{0}2^i$$
$$T = n - log_2n - 1 + log_2n$$
$$T = n - 1$$
Averaged, we get $T = O(1)$.

### Linked List based Stack
```C++
struct Node{
    int data;
    Node* next;
}

Node* top;

void push(int x){
    Node* temp = new Node();
    temp->data = x;
    temp->next = top;
    top = temp;
}

int pop(){
    if(top==NULL){
        cout<<"Already empty";
        return;
    }
    
    Node* temp = top;
    top = top->next;
    int result = temp->data;
    delete temp;
    return result;
}

bool isEmpty(){
    if(top==NULL)
        return true;
    else
        return false;
}
```

Linked List based stack implementation has the same push and pop time complexity as array based implementation (as long as array based implementation has fixed length).

### Infix Postfix and Prefix
A normal arithmetic expression comprises of opertor in between operands. Expression is evaluated from left to right and we have to take into consideration the precedence of various operators. Paranthesis is used to specify what is evaluated first. The expression `A + B * C + D` can be rewritten as `((A + (B * C)) + D)` to show that the multiplication happens first, followed by the leftmost addition.  

**Prefix:** notation requires that all operators precede the two operands that they work on  
**Postfix:** notation requires that its operators come after the corresponding operands  

Examples:
```
| Infix             | Prefix        | Postfix       |
|-------------------|---------------|---------------|
| A + B * C + D     | + + A * B C D | A B C * + D + |
| (A + B) * (C + D) | * + A B + C D | A B + C D + * |
| A * B + C * D     | + * A B * C D | A B * C D * + |
```

Prefix and postfix notations have the benefit of not requiring paranthesis to determine the order of operations.

**Converting infix to postfix and prefix:** if the infix operation is completely paranthesized, we traverse the expression and a) in case of postfix replace ')' with the operator b) in case of prefix replace '(' with the operator.

![Postfix](https://runestone.academy/runestone/books/published/pythonds/_images/moveright.png)
![Prefix](https://runestone.academy/runestone/books/published/pythonds/_images/moveleft.png)  

Infix to postfix:
```java
public static String convert(String input) {
    StringBuffer output = new StringBuffer();

    // Only operators are added to this stack
    Stack<String> operatorStack = new Stack<>();

    // Considering input has space separated symbols
    String[] tokens = input.split(" ");

    for(String token: tokens) {
        // if not operator, just append to output
        if(!(token.equals("+") || token.equals("-") ||
            token.equals("*") || token.equals("/") ||
            token.equals("(") || token.equals(")"))) {
            output.append(token);
        // add opening paranthesis to the stack
        } else if(token.equals("(")) {
            operatorStack.push("(");
        // on encountering closing paranthesis, remove
        // everything from stack (and append) till you
        // get to corresponding opening paranthesis
        } else if(token.equals(")")) {
            String popped = null;
            while(!operatorStack.isEmpty() && 
                  (popped = operatorStack.pop()) != "(") {
                output.append(popped);
            }
        // For other operators before inserting into stack
        // remove all operators which have equal or greater
        // precedence
        } else if(token.equals("+") || token.equals("-")) {
            while(!operatorStack.isEmpty()) {
                String top = operatorStack.peek();
                if(top.equals("+") || top.equals("-") ||
                   top.equals("*") || top.equals("/")) {
                    output.append(operatorStack.pop());
                } else {
                    break;
                }
            }
            operatorStack.push(token);
        } else if(token.equals("*") || token.equals("/")) {
            while(!operatorStack.isEmpty()) {
                String top = operatorStack.peek();
                if(top.equals("*") || top.equals("/")) {
                    output.append(operatorStack.pop());
                } else {
                    break;
                }
            }
            operatorStack.push(token);
        }
    }

    // Empty out stack
    while(!operatorStack.isEmpty()) {
        output.append(operatorStack.pop());
    }

    return new String(output);
}
```

Evaluating postfix expression
```java
public static int evaluate(String input) {
    // Only numbers are added to this stack
    Stack<Integer> numStack = new Stack<>();

    // Considering input has space separated symbols
    String[] tokens = input.trim().split(" ");

    for(String token: tokens) {
        // push numbers to stack
        if(!(token.equals("+") || token.equals("-") ||
             token.equals("*") || token.equals("/"))) {
            numStack.push(Integer.parseInt(token));
        } else {
            int rightOperand = numStack.pop();
            int leftOperand = numStack.pop();
            int result = 0;

            // evaluate binary expression as per operator
            // then push result to stack again
            if(token.equals("+")) {
                result = leftOperand + rightOperand;
            } else if(token.equals("-")) {
                result = leftOperand - rightOperand;
            } else if(token.equals("*")) {
                result = leftOperand * rightOperand;
            } else if(token.equals("/")) {
                result = leftOperand / rightOperand;
            }

            numStack.push(result);
        }
    }

    return numStack.pop();
}
```

### Built-in Implementations
**Java:**
- Using Stack  

```java
Stack<Integer> stack = new Stack<>();

// Checking if stack is empty
stack.isEmpty();

// Get size of stack
stack.size();

// Pushing items to stack
// add() adds to end of list
stack.add(5); stack.add(6);

// Popping items from stack
// pop() removes from end of list
// Throws EmptyStackException if stack is empty
int item = stack.pop();

// Iterating over stack
for(int i: stack){
    /* code */
}
```

- Using ArrayList  

```java
ArrayList<Integer> stack = new ArrayList<>();

// Checking if stack is empty
stack.isEmpty();

// Get size of stack
stack.size();

// Pushing items to stack
stack.add(5); stack.add(6);

// Popping items from stack
int item = stack.remove(stack.size() - 1);
```

**Python:**
```sh
# List is generally used as stack
stack = []

# Checking if stack is empty
len(stack) == 0

# Get size of stack
len(stack)

# Pushing items to stack
# append() adds to end of list
stack.append(5)

# Popping items from stack
# pop() removes from end of list
item = stack.pop()

# Iterating over stack
for i in stack:
    pass
```

**Q 1:** Implement a function `getMin()` which returns the minimum element in the stack.  
**Answer:** So get minimum element of stack, we need to make use of two stacks. One main stack and the other stack which holds current minimum at the top.

In [6]:
stack = []
# This stack contains all the minimum
# elements
min_stack = []
def push(x):
    stack.append(x)
    # Both stacks are empty, so append
    if len(stack) == 0:
        min_stack.append(x)
    else:
        # Check if the newly added element is less
        # than the current minimum element
        if x <= min_stack[-1]:
            min_stack.append(x)
            
def pop():
    popped = stack.pop()
    if popped == min_stack[-1]:
        min_stack.pop()
        
def get_min():
    return min_stack[-1]
        
push(7)
push(8)
push(3)
print(get_min())
push(1)
push(0)
push(0)
pop()
print(get_min())

3
0


How can we maintain minimum without using extra space? If all the numbers to be inserted into the stack are positive then we can follow the below steps. Consider the array is `6, 3, 7, 8, 2`. We insert 6 first into the stack. The variable `min_stack = 6`. Next when we push 3, `min_stack = 3`, however the value inserted into the stack will be `3 - min_stack` which is `3 - 6 = -3`. We will insert 7 and 8 as it is. When we insert 2, we make `min_stack = 2`, however we insert `2 - 3 = -1` into the stack. And so on. The stack now is `6, -3, 7, 8, -1`.  
Now when we pop elements and it is negative, then to get to the original number we need to follow the following equation: `x - prev_min_stack = top_of_stack`. But how to get the previous min from current min? `prev_min_stack = min_stack - top_of_stack`. `prev_min_stack = 2 - (-1) = 3`. So the popped elements is `x - 3 = -1 = 2`. Make `min_stack = prev_min_stack`.

**Q 2:** Sort a stack  
**Answer:**

In [2]:
stack = []

def sort():
    # Create aux stack
    aux = []
        
    while(len(stack) != 0):
        popped = stack.pop()
        # If aux is empty just push
        if len(aux) == 0:
            aux.append(popped)
            
        # If the element to be added to aux is greater
        # than or equal to element at top, push
        elif popped <= aux[-1]:
            aux.append(popped)
        # Pop out out elements greater before pushing
        else:
            while(len(aux) != 0 and aux[-1] < popped):
                stack.append(aux.pop())
            aux.append(popped)
            
    while(len(aux) != 0):
        stack.append(aux.pop())
            
stack = [2,3,1,5,1]
sort()
print(stack)

[1, 1, 2, 3, 5]


**Q 3:** Given a string, remove all the consequtive duplicate elements. For example, if the string is `abbck`, then return `ack`. However consider `kabbal`. It will convert to `kaal` and then finally to `kl`. Finally if we pass `aaa` as input, we should get `a` .  
**Answer:**

In [18]:
def remove_duplicate(input):
    stack = []
    for i in input:
        if len(stack) == 0:
            stack.append(i)
        else:
            # If the element to be inserted is already
            # present at the top, pop that element
            if stack[-1] == i:
                stack.pop()
            # Else push element to the stack
            else:
                stack.append(i)
    return ''.join(stack)

print(remove_duplicate('kabbal'))
print(remove_duplicate('aaa'))
print(remove_duplicate('abbck'))

kl
a
ack


**Q 4:** Determine whether the given set of paranthesis is valid or not. Paranthesis can be either of the following: `[, {, (, ), }, ]`  
**Answer:**

In [21]:
def valid_paranthesis(input):
    opening = ['{', '[', '(']
    closing = ['}', ']', ')']
    stack = []
    for i in input:
        if i in closing and len(stack) == 0:
            return False
        
        if i in opening:
            stack.append(i)
        else:
            popped = stack.pop()
            if (i == '}' and popped == '{') or (i == ']' and popped == '[') or (i == ')' and popped == '('):
                continue
            else:
                return False
            
    if len(stack) == 0:
        return True
    else:
        return False

par = '{[)(]}'                
print(valid_paranthesis(par))

par = '{[()]}'                
print(valid_paranthesis(par))

False
True


**Q 5:** Given an array, for each element find the closest number (to the right) greater than that number. For example, if the array is `[10, 7, 4, 2, 9, 10, 11, 3, 2]`, the answer would be `[11, 9, 9, 9, 10, 11, -1, -1, -1]` .   
**Answer:** 

In [8]:
def next_larger(A):
    i = len(A) - 1
    stack = []
    answer = [-1] * len(A)
    while(i >= 0):
        val = A[i]
        
        while(len(stack) != 0 and val >= stack[-1]):
            stack.pop()
            
        if len(stack) != 0:
            answer[i] = stack[-1]
        
        stack.append(val)
        i -= 1
    return answer

A = [10, 7, 4, 2, 9, 10, 11, 3, 2]
print(next_larger(A))

[11, 9, 9, 9, 10, 11, -1, -1, -1]


Similarly, previous larger would be

In [11]:
def prev_larger(A):
    i = 0
    stack = []
    answer = [-1] * len(A)
    while(i < len(A)):
        val = A[i]
        
        while(len(stack) != 0 and val >= stack[-1]):
            stack.pop()
            
        if len(stack) != 0:
            answer[i] = stack[-1]
        
        stack.append(val)
        i += 1
    return answer

A = [10, 7, 4, 2, 9, 10, 11, 3, 2]
print(prev_larger(A))

[-1, 10, 7, 4, 10, -1, -1, 11, 3]


In similar fashion,

In [12]:
def next_smaller(A):
    i = len(A) - 1
    stack = []
    answer = [-1] * len(A)
    while(i >= 0):
        val = A[i]
        
        while(len(stack) != 0 and val <= stack[-1]):
            stack.pop()
            
        if len(stack) != 0:
            answer[i] = stack[-1]
        
        stack.append(val)
        i -= 1
    return answer

A = [10, 7, 4, 2, 9, 10, 11, 3, 2]
print(next_smaller(A))

[7, 4, 2, -1, 3, 3, 3, 2, -1]


And,

In [13]:
def prev_smaller(A):
    i = 0
    stack = []
    answer = [-1] * len(A)
    while(i < len(A)):
        val = A[i]
        
        while(len(stack) != 0 and val <= stack[-1]):
            stack.pop()
            
        if len(stack) != 0:
            answer[i] = stack[-1]
        
        stack.append(val)
        i += 1
    return answer

A = [10, 7, 4, 2, 9, 10, 11, 3, 2]
print(prev_smaller(A))

[-1, -1, -1, -1, 2, 9, 10, 2, -1]
